# Imports

In [1]:
!pip install datasets -q

In [2]:
import json
from datasets import load_dataset
from tqdm import tqdm

In [3]:
mrobust_path = 'data/mrobust-data'


# mRobust

* https://huggingface.co/datasets/unicamp-dl/mrobust

## collection

In [33]:
!wc -l ../data/mrobust-pt/full_collection/raw-alpha-ids.tsv

  528032 ../data/mrobust-pt/full_collection/raw-alpha-ids.tsv


In [16]:
!head -n 5 ../data/mrobust-pt/full_collection/raw-alpha-ids.tsv

FT931-16655	930105 FT 05 JAN 93 / Redland concorda acordos europeus de 59 milhões de libras REDLAND, o grupo de materiais de construção que pagou 624 milhões de libras em ações pela Steetley no ano passado, arrecadou 100 milhões de libras por meio de alienações. Está reinvestindo 58,6 milhões de libras em suas atividades de azulejos e tijolos na Europa continental. A Redland está formando uma joint venture com a Koramic, uma empresa privada belga, que engloba as fábricas de tijolos das duas empresas na Holanda, Alemanha e Bélgica. A nova empresa será a maior fabricante de tijolos de fachada da Europa continental. Os lucros das atividades combinadas em 1992 teriam sido de 15,9 milhões de libras, dos quais a parte de Redland teria sido de 5,8 milhões de libras. A joint venture 50:50 deterá 70 por cento das atividades combinadas, dando a Redland uma participação econômica de 35 por cento, mas controle conjunto. Redland também receberá 17,3 milhões de libras em dinheiro. A Redland está aum

preprocess collection converting string ids to int

> from https://github.com/monilouise/unicamp-P_IA368DD_2023S1/blob/main/Final_project/Splade_v2_PT_indexing.ipynb thanks monique 🫶

In [22]:
map_id_to_text = dict()

with open('../data/mrobust-pt/full_collection/raw-alpha-ids.tsv', 'r') as file:
  for line in file:
    fields = line.split('\t')
    id = fields[0]
    text = fields[1]
    map_id_to_text[id] = text

In [25]:
doc_ids = map_id_to_text.keys()

In [30]:
map_id_to_int = {value:index for index, value in enumerate(doc_ids)}

In [31]:
new_map_id_to_text = {map_id_to_int[id]: text for id, text in map_id_to_text.items()}

In [32]:
len(new_map_id_to_text)

528032

In [34]:
with open('../data/mrobust-pt/full_collection/raw.tsv', 'w') as file:
  for id, text in new_map_id_to_text.items():
    file.write(f"{id}\t{text}")

In [35]:
!wc -l ../data/mrobust-pt/full_collection/raw.tsv

  528032 ../data/mrobust-pt/full_collection/raw.tsv


In [36]:
!head -n 5 ../data/mrobust-pt/full_collection/raw.tsv

0	930105 FT 05 JAN 93 / Redland concorda acordos europeus de 59 milhões de libras REDLAND, o grupo de materiais de construção que pagou 624 milhões de libras em ações pela Steetley no ano passado, arrecadou 100 milhões de libras por meio de alienações. Está reinvestindo 58,6 milhões de libras em suas atividades de azulejos e tijolos na Europa continental. A Redland está formando uma joint venture com a Koramic, uma empresa privada belga, que engloba as fábricas de tijolos das duas empresas na Holanda, Alemanha e Bélgica. A nova empresa será a maior fabricante de tijolos de fachada da Europa continental. Os lucros das atividades combinadas em 1992 teriam sido de 15,9 milhões de libras, dos quais a parte de Redland teria sido de 5,8 milhões de libras. A joint venture 50:50 deterá 70 por cento das atividades combinadas, dando a Redland uma participação econômica de 35 por cento, mas controle conjunto. Redland também receberá 17,3 milhões de libras em dinheiro. A Redland está aumentando su

## queries

In [8]:
!wc -l ../data/mrobust-pt/dev_queries/raw.tsv

     250 ../data/mrobust-pt/dev_queries/raw.tsv


In [17]:
!head -n 5 ../data/mrobust-pt/dev_queries/raw.tsv

301	Identifique as organizações que participam da atividade criminosa internacional, a atividade e, se possível, as organizações colaboradoras e os países envolvidos.
302	A doença da poliomielite (poliomielite) está sob controle no mundo?
303	Identifique realizações positivas do telescópio Hubble desde que foi lançado em 1991.
304	Faça uma lista de mamíferos considerados em perigo, identifique seu habitat e, se possível, especifique o que os ameaça.
305	Quais são os veículos de passageiros mais resistentes a choques e menos resistentes a choques?


## qrel

In [14]:
!wc -l ../data/mrobust-pt/qrel/qrels.robust04.txt

  311410 ../data/mrobust-pt/qrel/qrels.robust04.txt


In [18]:
!head -n 5 ../data/mrobust-pt/qrel/qrels.robust04.txt

301 0 FBIS3-10082 1
301 0 FBIS3-10169 0
301 0 FBIS3-10243 1
301 0 FBIS3-10319 0
301 0 FBIS3-10397 1


In [19]:
with open('../data/mrobust-pt/qrel/qrels.robust04.txt', 'r') as fin, \
     open('../data/mrobust-pt/qrel/qrel.tsv', 'w', newline='') as fout:
        for line in fin:
            fout.write(line.replace(" ", "\t"))     

In [20]:
!wc -l ../data/mrobust-pt/qrel/qrel.tsv

  311410 ../data/mrobust-pt/qrel/qrel.tsv


In [21]:
!head -n 5 ../data/mrobust-pt/qrel/qrel.tsv

301	0	FBIS3-10082	1
301	0	FBIS3-10169	0
301	0	FBIS3-10243	1
301	0	FBIS3-10319	0
301	0	FBIS3-10397	1


In [39]:
from collections import defaultdict
import json
import os

def build_json_qrel(qrel_file_path):
    temp_d = defaultdict(dict)
    with open(qrel_file_path) as reader:
        for line in reader:
            q_id, _, d_id, rel = line.split("\t")
            if d_id in map_id_to_int:
                doc_id = map_id_to_int[d_id]            
                temp_d[q_id][doc_id] = int(rel)
    print(f"built qrel file, contains {len(temp_d)} queries...")
    json.dump(dict(temp_d), open(os.path.join(os.path.dirname(qrel_file_path), "qrel.json"), "w"))

In [40]:
build_json_qrel("../data/mrobust-pt/qrel/qrel.tsv")

built qrel file, contains 249 queries...
